### Prerequisites

After compiling the DFT driver and installing PyCDFT, run the ground state calculation.
- - - - - - -
For Qbox

```bash
 export qb="/path/to/executable"
 $qb < gs.in > gs.out
```

Then in the same directory, run [Qbox in server mode](qboxcode.org/daoc/html/usage/client-server.html)

```bash
 mpirun -np <ntasks> $qb -server qb_cdft.in qb_cdft.out
```

where qb_cdft.\* are files reserved in client-server mode.

### Tutorial: constrained forces on He$_2^+$

In this tutorial, we explore the calculation of forces for a simple diatomic He$_2^+$ molecule as a function of He-He distance

In [25]:
from pycdft import *
from ase.io import read
import numpy as np

In [22]:
cell = read("./He2.cif")
print(r"Initial atomic positions (Ang):")
print(cell.get_positions())

Initial atomic positions (Ang):
[[ 0.        0.        0.      ]
 [ 0.        0.        1.587525]]


In [27]:
# vary distance of He atoms, in Angstroms
# need check if intended to use these numbers in Angstroms
# left here in order to match with ref-he2_forces.ipynb
ang2bohr= 0.529177
ds = np.multiply([2.9, 3.0, 3.1],ang2bohr)
# and associated initial guess of Lagrange multiplier, in eV (?)
Vs = [1.191584, 1.274559, 1.329482]

Eds = []
Ecs = []
Ws = []
Fds = []
Fcs = []
Fws = []

for d, V in zip(ds, Vs):
    #cell.atoms[1].abs_coord[2] = d
    # sample = Sample(ase_cell=cell.ase_cell, n1=80, n2=80, n3=80, vspin=1)
    cell.positions[1][2] =d 
    sample = Sample(ase_cell=cell, n1=80, n2=80, n3=80, vspin=1)
    print(sample.atoms[1])
    
    # load DFT driver
    qboxdriver = QboxDriver(
        sample=sample,
        init_cmd="load gs.xml\nset xc PBE\nset wf_dyn PSDA\nset_atoms_dyn CG\nset scf_tol 1.0E-8\n",
        scf_cmd="run 0 50 5",
    )
    
    # set up CDFT constraints and solver
    solver = CDFTSolver(job="opt", optimizer="secant", sample=sample, maxstep=1, dft_driver=qboxdriver)
    
    c = ChargeConstraint(
        sample=solver.sample,
        fragment=Fragment(solver.sample, solver.sample.atoms[0:1]),
        V_init=V,
        N0=1,
        N_tol=1.0E-3
    )
#     c = ChargeTransferConstraint(
#         sample=solver.sample,
#         donor=Fragment(solver.sample, solver.sample.atoms[0:1]),
#         acceptor=Fragment(solver.sample, solver.sample.atoms[1:2]),
#         V_init=0,
#         N0=1,
#         N_tol=1.0E-3
#     )
    
    solver.solve()
    
    # collect energies and forces
    Eds.append(sample.Ed)
    Ecs.append(sample.Ec)
    Ws.append(sample.W)
    Fds.append(sample.Fd[0][2])
    Fcs.append(sample.Fc[0][2])
    Fws.append(sample.Fw[0][2])

Atom He. Abs coord (0.000, 0.000, 2.900). 
QboxDriver: setting output path to ./pycdft_outputs/solver4/...
QboxDriver: waiting for Qbox to start...
QboxDriver: initializing Qbox...
Geometry optimization step 1
Updating constraint with new structure...
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
SCF iteration 1
  Ed (DFT energy) = -4.723272
  Ec (constraint energy) = 1.191470
  E (Ed + Ec) = -3.531802
  W (free energy) = -4.723256
  > Constraint #0 (type = charge, N0 = 1, V = 1.191584):
    N = 0.999891
    dW/dV = N - N0 = -0.000109
~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
*Constrained SCF converged!*

Maximum force = 0.207179 au, on 1th atom (He).  Fw = 0.000000, 0.000000, 0.207179
  Fd = -0.000000, -0.000000, 0.121503; Fc = 0.000000, 0.000000, 0.085676
Updating constraint with new structure...

**Constrained optimization NOT achieved after 1 steps!**

Atom He. Abs coord (0.000, 0.000, 3.000). 
QboxDriver: setting output path to ./pycdft_outputs/solver5/...
QboxDriver: waiting for Qbox to start...
QboxD

### Analysis

Now let's take a look at some of the outputs.

Eds = DFT energy, without constraint potential

Ecs = constrained energy corresponding to $\sum_k V_k \sum_\sigma \int w_k^\sigma(\boldsymbol{r})n^\sigma(\boldsymbol{r})d^3\boldsymbol{r}$

Ws = free energies, Ws == Eds upon convergence

Fds, Fcs, Fws = corresponding forces; Fd is Qbox output; Fw = Fd + Fc used for geometry optimization calculations

Qbox is in atomic units (Ry, Bohr, ...)

In [28]:
Eds

[-4.7232715800000005, -4.7156742000000005, -4.705947500000001]

In [29]:
Ecs

[1.1914696, 1.27583943, 1.3307623]

In [30]:
Ws

[-4.7232563999255799, -4.7156573929960706, -4.7059306209158631]

In [31]:
Fds

[0.057669959999999999, 0.05349098, 0.04715403]

In [32]:
Fcs

[0.09388839800073244, 0.10086640503002132, 0.10679375870302286]

In [33]:
Fws

[0.15155835800073245, 0.15435738503002133, 0.15394778870302286]

In [34]:
%matplotlib inline
import matplotlib.pyplot as plt

In [35]:
Ets = np.array(Eds) + Ecs

In [36]:
Ets - np.array(Vs) * 1.2

array([-4.96170278, -4.96930557, -4.9705636 ])

In [ ]:
Ets = np.array(Eds) + np.array(Ecs)

Ed = Eds[1]
Ec = Ecs[1]
Et = Ets[1]
W = Ws[1]

Fd = Fds[1]
Fc = Fcs[1]
Fw = Fws[1]

In [ ]:
Eds_by_force = [Ed - (-0.1) * (-Fd), Ed, Ed - (0.1) * (-Fd)]
Ecs_by_force = [Ec - (-0.1) * (-Fc), Ec, Ec - (0.1) * (-Fc)]
Ets_by_force = [Et - (-0.1) * (-Fw), Et, Et - (0.1) * (-Fw)]
Ws_by_force = [W - (-0.1) * (-Fw), W, W - (0.1) * (-Fw)]

In [ ]:
plt.plot(ds, Eds)
plt.plot(ds, Eds_by_force, "--")

In [ ]:
plt.plot(ds, Ecs)
plt.plot(ds, Ecs_by_force, "--")

In [ ]:
plt.plot(ds, Ets)
plt.plot(ds, Ets_by_force, "--")

In [ ]:
plt.plot(ds, Ws)
plt.plot(ds, Ws_by_force, "--")